In [3]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import matplotlib.image as mpimg
from shapely.geometry import Point
from osgeo import gdal
from osgeo import ogr
import rasterio
import itertools
import math
from rasterio.plot import show
from pyproj import CRS
from matplotlib.ticker import FuncFormatter, MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator, LinearLocator
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import scipy.interpolate as interpolate
import csv
import cv2
import fiona
from shapely.geometry import LineString, mapping
from scipy.interpolate import splprep, splev
from skimage.measure import label, regionprops
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy.interpolate import splprep, splev, UnivariateSpline
from shapely.geometry import LineString
import ipywidgets as widgets
from ipywidgets import interact
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import Normalize
from matplotlib.dates import DateFormatter, HourLocator
from datetime import datetime, timedelta
import matplotlib.colors as mcolors  # Importing colors specifically
from matplotlib import cm
import matplotlib.cm as cm
import textwrap
import os
import matplotlib.ticker as mticker
import numpy.ma as ma
from rasterio.plot import plotting_extent
from skimage.color import label2rgb
from skimage.draw import line
from scipy.signal import find_peaks
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
from skimage import exposure
from skimage.morphology import binary_closing, binary_dilation, skeletonize, disk
from skimage import morphology, measure, feature
from skimage.morphology import skeletonize
from scipy.ndimage import binary_erosion
plt.rcParams['savefig.dpi'] = 500
plt.rc('font', family='Times New Roman')
plt.rcParams.update({'font.size': 25})
from matplotlib.ticker import FuncFormatter, MaxNLocator, ScalarFormatter
import matplotlib.dates as mdates
%matplotlib qt

PLOT FULL SAR IMAGE

In [7]:

SAR_image = 'Site_Image.tif'

with rasterio.open(SAR_image) as src:
    # Read band 2
    sar_img = src.read(2)
    transform = src.transform
    nodata = src.nodata
    crs = src.crs

# Create a masked array for nodata pixels if nodata is defined
if nodata is not None:
    sar_masked = ma.masked_equal(sar_img, nodata)
else:
    sar_masked = sar_img

# Exclude nodata pixels for dynamic range
if nodata is not None:
    valid_data = sar_img[sar_img != nodata]
else:
    valid_data = sar_img

# Determine dynamic range from valid pixels
dB_min = valid_data.min()
dB_max = valid_data.max()

# Compute the spatial extent for a georeferenced plot
extent = plotting_extent(src)

fig, ax = plt.subplots(figsize=(9, 8))

# Plot the masked array
im = ax.imshow(sar_masked, cmap='gray', vmin=dB_min, vmax=dB_max,
               extent=extent, origin='upper')

ax.set_xlabel('Eastings (m)')
ax.set_ylabel('Northings (m)')
ax.set_title("Flagler Beach")
# ax.axis('off')
cb = fig.colorbar(im, ax=ax, label='Backscatter (dB)')

# Format axes ticks to plain style, zero decimals
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

plt.show()


READ THE SAR IMAGE (CLIPPED TO AREA OF INETEREST) AND CONVERT TO BINARY IMAGE (0,1) WHERE PIXELS > THRESHOLD = 1
For this example, threshold value = -15.54

In [9]:
#######################################################################################################
# THIS CODE INTERACTIVELY CONVERTS THE IMPUT SAR IMAGE TO A BINARY IMAGE BASED ON A SELECTED THRESHOLD.
# PIXEL BACKSCATTER > THE THRESHOLD ARE SET TO 1 OTHERWISE 0. 
# THE USER SHOULD SELECT THE THRESHOLD WHICH PROVIDES THE BEST GEOMETRY OF CUSPS HORNS. 
# USING THE SLIDER MAKES THRESHOLDING EASIER
#######################################################################################################

print("Drag the slider below to choose the threshold. Pixels with dB > threshold become 1, else 0.\n")

input_file = 'Cusps_Area_Image.tif'
with rasterio.open(input_file) as src:
    img = src.read(2)  # SAR image in dB
    nodata = src.nodata

if nodata is not None:
    valid_mask = img != nodata
    valid_data = img[valid_mask]
else:
    valid_mask = np.ones_like(img, dtype=bool)
    valid_data = img

print("Valid pixel statistics (excluding nodata):")
print("Min: {:.2f} dB, Max: {:.2f} dB, Mean: {:.2f} dB, Std: {:.2f} dB".format(
    valid_data.min(), valid_data.max(), valid_data.mean(), valid_data.std()))

# Use the valid data range as our dynamic range.
dB_min = valid_data.min()
dB_max = valid_data.max()


global_binary_mask = None  # Declare a global variable

# Create a new figure for binary mask thresholding.
fig2, ax_bin = plt.subplots(figsize=(8, 8))
plt.tight_layout()

def update_binary_mask(binary_thresh):
    global global_binary_mask
    binary_mask = (img > binary_thresh).astype(np.uint8)
    
    # Save the binary mask to the global variable.
    global_binary_mask = binary_mask.copy()
    
    ax_bin.cla()
    ax_bin.imshow(binary_mask, cmap='gray')
    ax_bin.set_title("Binary Mask (Threshold = {:.2f} dB)".format(binary_thresh))
    ax_bin.axis('off')

    # Adjust the top margin
    # fig2.subplots_adjust(top=0.88)
    fig2.canvas.draw_idle()
    
interact(update_binary_mask,
         binary_thresh=widgets.FloatSlider(min=dB_min, max=dB_max, step=0.01, value=-18, description="Binary Threshold"));

Drag the slider below to choose the threshold. Pixels with dB > threshold become 1, else 0.

Valid pixel statistics (excluding nodata):
Min: -32.74 dB, Max: -5.46 dB, Mean: -17.25 dB, Std: 3.08 dB


interactive(children=(FloatSlider(value=-18.0, description='Binary Threshold', max=-5.463332176208496, min=-32…

In [ ]:
PROCESS BINARY IMAGE TO DEFINE CUSPS GEOMETRY

In [13]:
# NOTE !!!!!!!
# -----------------------------------------------------------------------------------------------
# THIS CELL REMOVES SMALL UNWANTED REGIONS AND PERFORMS REGION FILLING ON THE BINARY MASK.
# ALL REGIONS ARE THEN IDENTIFIED WITH THEIR REGION ID AND ASSIGNED A COLOR.
# REGIONS OF INTEREST THAT ARE ISOLATED MAY BE NOTED AND SPECIFICALLY FILTERED IN THE NEXT CELL
# --------------------------------------------------------------------------------------

######################################################################
# 1) MORPHOLOGICAL CLOSING AND REGION FILLING
#####################################################################
try:
    binary_mask = global_binary_mask
except NameError:
    raise NameError("global_binary_mask is not defined. Please run the binary mask interactive cell first.")

# (a) Filter out small isolated regions
labeled_mask_init = label(binary_mask)
filtered_mask = np.zeros_like(binary_mask, dtype=np.uint8)
min_area_1 = 20  # Adjust as needed
for region in regionprops(labeled_mask_init):
    if region.area >= min_area_1:
        filtered_mask[tuple(region.coords.T)] = 1

# (b) First morphological closing
selem = disk(5)
binary_mask_closed = binary_closing(filtered_mask, selem)

# (c) Label the closed mask
labeled_mask_closed = label(binary_mask_closed)
max_label = labeled_mask_closed.max()

#####################################################################
# 2) IDENTIFY EACH REGION IN THE MASK AND ASSIGN A DISTINCT COLOR
#####################################################################
# We'll assign a distinct color for each region ID from 1..max_label.
# Region 0 (background) can be transparent or black.

colors = [(0, 0, 0, 0)]  # RGBA transparent for background label=0
for i in range(1, max_label + 1):
    frac = i / (max_label + 1)
    color = plt.cm.tab20(frac)  # sample from tab20 colormap
    colors.append(color)

# Create a ListedColormap and a BoundaryNorm
cmap = mcolors.ListedColormap(colors)
norm = mcolors.BoundaryNorm(range(max_label + 2), cmap.N)

#####################################################################
# 3) Visualization with a 1×3 subplot
#####################################################################
fig, axes = plt.subplots(1, 3, figsize=(15, 10))

# Subplot 1: original binary_mask
axes[0].imshow(binary_mask, cmap='gray', origin='upper')
axes[0].set_title(textwrap.fill("Original Binary Mask", width=20), fontsize=25)
axes[0].axis('off')

# Subplot 2: filtered_mask (isolated regions removed)
axes[1].imshow(filtered_mask, cmap='gray', origin='upper')
axes[1].set_title(textwrap.fill("Filtered Mask (Isolated Pixels Removed)", width=25), fontsize=25)
axes[1].axis('off')

# Subplot 3: color-labeled mask after morphological closing
axes[2].imshow(labeled_mask_closed, cmap=cmap, norm=norm, origin='upper')
axes[2].set_title(textwrap.fill("Modified: Regions filled", width=25), fontsize=25)
axes[2].axis('off')

# Build legend entries for each region label 1..max_label
patches = []
for label_id in range(1, max_label + 1):
    color = cmap(norm(label_id))
    patch = mpatches.Patch(color=color, label=f"Region {label_id}")
    patches.append(patch)
axes[2].legend(handles=patches, loc='center left', bbox_to_anchor=(1, 0.5), borderaxespad=0.4, fontsize=25)

plt.tight_layout()
plt.subplots_adjust(wspace=.3)
plt.show()

# Print out region indices and info
props_closed = regionprops(labeled_mask_closed)
print("Regions after morphological closing:")
for region in props_closed:
    print(f" Region {region.label}: area={region.area}, bbox={region.bbox}")

print("\nUse the legend above to see which color corresponds to which Region ID.")
print("You can now pick one or multiple region IDs to keep in the next steps.\n")
print("\nOr You may now decide if you want to delete any region(s). " 
      "Run the next cell to proceed with optional deletions.\n")

Regions after morphological closing:
 Region 1: area=492.0, bbox=(10, 7, 55, 32)
 Region 2: area=6922.0, bbox=(62, 18, 365, 153)
 Region 3: area=23.0, bbox=(139, 91, 146, 95)
 Region 4: area=141.0, bbox=(325, 104, 374, 122)

Use the legend above to see which color corresponds to which Region ID.
You can now pick one or multiple region IDs to keep in the next steps.


Or You may now decide if you want to delete any region(s). Run the next cell to proceed with optional deletions.



(OPTIONAL) DELETE REGIONS/PIXELS 

In [15]:
# ----------------------------------------------
# EXECUTIVE THIS CELL IF YOU NEED TO DELETE SOME REGION(S) FROM THE MASK
# ----------------------------------------------

# NOTE !! INTERACTIVE CODE : CHECK OUTPUT FOR INSTRUCTIONS
#####################################################################
# DELETE UNWANTED REGIONS BY KEEPING ONLY SPECIFIED REGION(S)
# SELECT THE REGION ID YOU WANT TO KEEP
#####################################################################
# We'll reuse labeled_mask_closed, props_closed, binary_mask, filtered_mask from the previous cell.

# Instead of deleting, we ask the user to input which region IDs they want to KEEP.
keep_decision = input("Do you want to KEEP only specific region(s)? (y/n): ").strip().lower()
if keep_decision == 'y':
    kept_input = input("Enter region label(s) to KEEP (e.g. '2,4') or leave blank to skip: ").strip()
    if kept_input:
        kept_labels = [int(x.strip()) for x in kept_input.split(',')]
        updated_mask = np.zeros_like(labeled_mask_closed, dtype=np.uint8)
        
        # Keep only the regions whose label is in kept_labels
        for region in props_closed:
            if region.label in kept_labels:
                coords = region.coords
                updated_mask[coords[:,0], coords[:,1]] = 1
        
        # Re-label after selection
        final_labeled = label(updated_mask)
        max_label2 = final_labeled.max()
        
        # Build a new color-labeled version for the final mask
        colors2 = [(0,0,0,0)]
        for i in range(1, max_label2 + 1):
            frac = i / (max_label2 + 1)
            color = plt.cm.tab20(frac)
            colors2.append(color)
        
        cmap2 = mcolors.ListedColormap(colors2)
        norm2 = mcolors.BoundaryNorm(range(max_label2 + 2), cmap2.N)
        
        # 1×4 final figure for comparison
        fig2, ax2 = plt.subplots(1, 4, figsize=(18,5))
        
        ax2[0].imshow(binary_mask, cmap='gray', origin='upper')
        ax2[0].set_title("Original Binary Mask")
        ax2[0].axis('off')
        
        ax2[1].imshow(filtered_mask, cmap='gray', origin='upper')
        ax2[1].set_title("Filtered Mask")
        ax2[1].axis('off')
        
        ax2[2].imshow(labeled_mask_closed, cmap=cmap, norm=norm2, origin='upper')
        ax2[2].set_title("Before Selection")
        ax2[2].axis('off')
        
        ax2[3].imshow(final_labeled, cmap=cmap2, norm=norm2, origin='upper')
        ax2[3].set_title("After Selection")
        ax2[3].axis('off')
        
        patches2 = []
        for label_id in range(1, max_label2 + 1):
            color = cmap2(norm2(label_id))
            patch = mpatches.Patch(color=color, label=f"Region {label_id}")
            patches2.append(patch)
        ax2[3].legend(handles=patches2, loc='best')
        
        plt.tight_layout()
        plt.show()
        
        print("Final Regions after selection:")
        props_final = regionprops(final_labeled)
        for region in props_final:
            print(f" Region {region.label}: area={region.area}, bbox={region.bbox}")
        
        # Update the global mask so subsequent workflow sees the final mask.
        labeled_mask_closed = final_labeled
    else:
        print("No region labels entered. Skipping selection.")
else:
    print("No region selection made. 'labeled_mask_closed' remains unchanged.")

print("Done. 'labeled_mask_closed' now holds the final labeled mask.")


Do you want to KEEP only specific region(s)? (y/n):  y
Enter region label(s) to KEEP (e.g. '2,4') or leave blank to skip:  1,2


Final Regions after selection:
 Region 1: area=492.0, bbox=(10, 7, 55, 32)
 Region 2: area=6922.0, bbox=(62, 18, 365, 153)
Done. 'labeled_mask_closed' now holds the final labeled mask.


REGION BY REGION MORPHOLOGICAL OPERATION

In [17]:
# NOTE !! INTERACTIVE CODE : CHECK OUTPUT FOR INSTRUCTIONS
#############################################################################################################
# THIS CELL ISOLATES SPECIFIC REGIONS AND PERFORM MORPHOLOGICAL FILTERING AND RE-JOINS THEM TO THE BINARY MASK
# THE USER CAN SELECT SINGLE OR MULTIPLE REGIONS. USER HAS TO INDICATE REGION(S) OF INTEREST.
# USER INTERACTIVELY INDICATES WHAT MORPHOLOGICAL FILTERS TO USE (COULD BE EROSION, DILATION OR CLOSING) AND ASSOCIATED DISK SIZE
#############################################################################################################

def apply_ops_interactively(region_mask):
    """
    Interactively ask the user for morphological operations 
    (erosion, dilation, closing) and disk size, then apply them in order.
    region_mask: binary 2D array (0/1) for the region.
    Returns the updated region_mask (also binary 0/1).
    """
    # Prompt user for morphological ops
    print("Which morphological ops do you want to apply to this region?")
    print("Type them in a comma-separated list and press 'Enter', e.g. 'erosion,dilation,closing' or 'none'.")
    input_ops = input("Ops: ").strip().lower()
    if not input_ops or input_ops in ['none', 'no', 'skip']:
        print("No operations chosen. Keeping region as is.\n")
        return region_mask
    
    # Parse ops list
    ops_list = [op.strip() for op in input_ops.split(',')]
    
    # Prompt user for disk size
    disk_size_str = input("Enter disk size (integer) and press 'Enter', e.g. '3': ").strip()
    try:
        disk_size = int(disk_size_str)
    except ValueError:
        disk_size = 3
        print("Invalid disk size, defaulting to 3.")
    
    selem = disk(disk_size)
    updated = region_mask.copy()
    
    for op in ops_list:
        if op == 'erosion':
            updated = binary_erosion(updated, selem)
        elif op == 'dilation':
            updated = binary_dilation(updated, selem)
        elif op == 'closing':
            updated = binary_closing(updated, selem)
        else:
            print(f"Unknown operation '{op}' - skipping.")
    print(f"Applied ops={ops_list} with disk_size={disk_size}.\n")
    return updated

def interactive_modify_regions(labeled_mask, chosen_idxs, min_area=20):
    """
    1) For each region in chosen_idxs, isolate it, ask the user for morphological filters,
       apply them, remove small areas < min_area, then re-merge into the main mask.
    2) Return the updated binary mask (0/1).
    
    labeled_mask: labeled 2D array from a previous step.
    chosen_idxs: list of integer region labels to modify.
    min_area: area threshold for filtering out small objects after morphological ops.
    """
    # Convert the entire labeled mask to binary
    full_binary_mask = (labeled_mask > 0).astype(np.uint8)
    
    # If user picks no region, we skip modifications.
    if not chosen_idxs:
        print("No regions selected. Returning original mask.")
        return full_binary_mask
    
    # Gather regionprops for referencing
    props = regionprops(labeled_mask)
    
    # For each region in chosen_idxs, isolate, apply user-chosen filters, and re-merge
    for reg_label in chosen_idxs:
        # Find that region in props
        found = False
        for region in props:
            if region.label == reg_label:
                found = True
                coords = region.coords
                break
        if not found:
            print(f"Region label {reg_label} not found in the mask. Skipping.\n")
            continue
        
        # Isolate the region as a binary mask
        selected_mask = np.zeros_like(full_binary_mask, dtype=np.uint8)
        selected_mask[coords[:,0], coords[:,1]] = 1
        
        print(f"\n--- Region {reg_label} ---")
        print(f"Area={region.area}, BBox={region.bbox}.")
        
        # Ask user for morphological ops & disk size
        updated_region = apply_ops_interactively(selected_mask)
        
        # Filter out small areas again if needed
        labeled_region = label(updated_region)
        final_region = np.zeros_like(updated_region, dtype=np.uint8)
        for r in regionprops(labeled_region):
            if r.area >= min_area:
                final_region[tuple(r.coords.T)] = 1
        
        # Re-merge: remove old region from full_binary_mask
        full_binary_mask[coords[:,0], coords[:,1]] = 0
        # Add in the new updated region
        full_binary_mask[final_region == 1] = 1
    
    return full_binary_mask


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################################################################################################
# CODE IMPLEMENTATION
# ---------------------------------------------------------------

# Assume that 'labeled_mask_closed' is available from the previous cell
# Here, we re-label the closed mask to ensure consistency.
refined_labeled_mask_closed = label(labeled_mask_closed)

# SELECT THE REGION(S) OF INTEREST. ONE CAN SELECT SINGLE OR MULTIPLE REGIONS.
# For example, if you want to modify region 1, set chosen_regions = [1]
chosen_regions = [1,2]   # input the region(s) you want to apply morph filters to

# EXECUTE FUNCTION TO MODIFY INDIVIDUAL REGIONS INTERACTIVELY
full_binary_mask = interactive_modify_regions(refined_labeled_mask_closed, chosen_regions, min_area=20)

##################################################################################################
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Visualization
import textwrap
fig, ax = plt.subplots(1, 2, figsize=(12,8))
ax[0].imshow((labeled_mask_closed > 0).astype(np.uint8), cmap='gray')
ax[0].set_title(textwrap.fill("Before Region-by-Region Interactive Filtering", width=25))
ax[0].axis('off')

ax[1].imshow(full_binary_mask, cmap='gray')
ax[1].set_title(textwrap.fill("After Region-by-Region Interactive Filtering", width=25))
ax[1].axis('off')

fig.subplots_adjust(top=0.88)
plt.show()



--- Region 1 ---
Area=492.0, BBox=(10, 7, 55, 32).
Which morphological ops do you want to apply to this region?
Type them in a comma-separated list and press 'Enter', e.g. 'erosion,dilation,closing' or 'none'.


Ops:  dilation
Enter disk size (integer) and press 'Enter', e.g. '3':  3


Applied ops=['dilation'] with disk_size=3.


--- Region 2 ---
Area=6922.0, BBox=(62, 18, 365, 153).
Which morphological ops do you want to apply to this region?
Type them in a comma-separated list and press 'Enter', e.g. 'erosion,dilation,closing' or 'none'.


Ops:  closing
Enter disk size (integer) and press 'Enter', e.g. '3':  10


Applied ops=['closing'] with disk_size=10.



MERGE REGIONS

In [19]:
###########################################################################
# THIS CODE MERGES REGIONS FROM THE PREVIOUS CELL INTO ONE REGION FOR EASY DEFINITION OF CUSPS PROFILE
# IF ONLY 1 REGION IS PRESENT FROM THE PREVIOUS STEP, THIS WHOLE STEP IS SKIPPED
###########################################################################

def is_boundary_pixel(labeled_mask, r, c, label_id):
    """
    Check if pixel (r,c) with label_id is on the boundary by examining 8 neighbors.
    If any neighbor is background (0) or a different label, it's a boundary pixel.
    """
    rows, cols = labeled_mask.shape
    for dr in [-1, 0, 1]:
        for dc in [-1, 0, 1]:
            if dr == 0 and dc == 0:
                continue
            rr, cc = r + dr, c + dc
            if rr < 0 or rr >= rows or cc < 0 or cc >= cols:
                return True  # out of bounds => boundary
            if labeled_mask[rr, cc] != label_id:
                return True
    return False

def draw_thick_line(updated_mask, rA, cA, rB, cB, labelA, thickness=1):
    """
    Draw a 'thick' line of radius 'thickness' between (rA, cA) and (rB, cB) in updated_mask,
    setting those pixels to labelA.
    """
    rr, cc = line(rA, cA, rB, cB)
    rows, cols = updated_mask.shape
    # For each pixel in the line, fill a small disk
    for (r, c) in zip(rr, cc):
        for dr in range(-thickness, thickness+1):
            for dc in range(-thickness, thickness+1):
                if dr*dr + dc*dc <= thickness*thickness:
                    rr2 = r + dr
                    cc2 = c + dc
                    if 0 <= rr2 < rows and 0 <= cc2 < cols:
                        updated_mask[rr2, cc2] = labelA

def join_two_regions_by_line(labeled_mask, labelA, labelB, thickness=1):
    """
    Physically connect region labelA and labelB by drawing a thick line between
    the closest boundary pixels. Returns a copy of labeled_mask with that line
    set to labelA (or any desired label).
    """
    updated_mask = labeled_mask.copy()
    
    # Extract all pixels for region A and B
    coordsA = np.column_stack(np.where(updated_mask == labelA))
    coordsB = np.column_stack(np.where(updated_mask == labelB))
    
    if coordsA.size == 0 or coordsB.size == 0:
        print(f"Either region {labelA} or {labelB} does not exist in the mask.")
        return updated_mask
    
    # Find boundary pixels for A and B
    boundaryA = []
    boundaryB = []
    for (rA, cA) in coordsA:
        if is_boundary_pixel(updated_mask, rA, cA, labelA):
            boundaryA.append((rA, cA))
    for (rB, cB) in coordsB:
        if is_boundary_pixel(updated_mask, rB, cB, labelB):
            boundaryB.append((rB, cB))
    
    if not boundaryA or not boundaryB:
        print(f"No boundary pixels found for region {labelA} or {labelB}.")
        return updated_mask
    
    # Find pair of boundary pixels with minimal distance
    import math
    min_dist = math.inf
    bestA = None
    bestB = None
    for (rA, cA) in boundaryA:
        for (rB, cB) in boundaryB:
            dx = cB - cA
            dy = rB - rA
            dist_sqr = dx*dx + dy*dy
            if dist_sqr < min_dist:
                min_dist = dist_sqr
                bestA = (rA, cA)
                bestB = (rB, cB)
    
    # Draw a thick line between bestA and bestB, set to labelA
    if bestA and bestB:
        (rA, cA) = bestA
        (rB, cB) = bestB
        draw_thick_line(updated_mask, rA, cA, rB, cB, labelA, thickness=thickness)
    
    return updated_mask

def join_multiple_regions(labeled_mask, region_labels, thickness=1):
    """
    Join 2 or more region labels in 'region_labels' pairwise, unifying them under the first label.
    1) If there's only 1 region in the entire mask, skip merging entirely.
    2) If 'region_labels' has fewer than 2 labels, do nothing.
    3) Otherwise, do pairwise bridging in ascending order, merging each region into the first.
    Returns the final updated mask with those region labels joined into one region.
    """
    # Check how many total regions are in the entire mask
    all_labels = np.unique(labeled_mask)
    # If there's only one region in the entire mask (besides 0), skip
    nonzero_labels = all_labels[all_labels != 0]
    if len(nonzero_labels) <= 1:
        print("Only one region in the entire mask. No merging needed.")
        return labeled_mask
    
    # If the user-specified region_labels < 2, do nothing
    if len(region_labels) < 2:
        print("Need at least 2 region labels to join. Skipping.")
        return labeled_mask
    
    updated_mask = labeled_mask.copy()
    
    # Sort region_labels so we unify them in ascending order
    region_labels_sorted = sorted(region_labels)
    base_label = region_labels_sorted[0]  # we unify everything under the first label
    
    import math
    
    # Pairwise bridging: for each label in region_labels_sorted[1..end], bridge to base_label
    for lbl in region_labels_sorted[1:]:
        # 1) Bridge base_label and lbl
        updated_mask = join_two_regions_by_line(updated_mask, base_label, lbl, thickness=thickness)
        
        # 2) Merge lbl => base_label
        updated_mask[updated_mask == lbl] = base_label
        
        # 3) Re-label so that subsequent bridging sees a coherent mask
        updated_mask = label(updated_mask)
        # Possibly re-assign base_label if it changed. Let's find the largest region with the old base_label?
        # Or keep the same integer if we want a stable label. We'll do the simpler approach:
        # just find the new label that has the largest overlap with the old base_label coords
        old_coords = np.column_stack(np.where(labeled_mask == base_label))
        # but that might be complicated. Instead, let's not re-label every iteration:
        # We'll skip the re-label step if we want a stable label. 
        # However, we want to keep the bridging. We'll keep it simpler:
        # We'll do a partial approach: just unify them in the array, skip re-label each time
        # We'll do the re-label at the very end.
    
    # Finally, do one re-label pass to get a coherent final labeling
    updated_mask = label(updated_mask)
    return updated_mask

# ______________________________________________________________________________________________________________



# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################################################################################################
# CODE IMPLEMENTATION
# ---------------------------------------------------------------
labeled_full_binary_mask = label(full_binary_mask) # LABEL MASK FROM PREVIOUS CELL

# INDICATE THE REGIONS TO JOIN. CAN BE TWO OR MORE. IF ONLY ONE REGION IS IDENTIFIED/ENTERED, THE CODE SKIPS THIS WHOLE STEP
region_labels_to_join = [1,2]

# EXECUTE FUNCTION TO JOIN SELECTED REGIONS. THICKNESS INDICATES THE THICKNESS OF THE PIXEL USED FOR JOINING THE REGIONS
final_mask = join_multiple_regions(labeled_full_binary_mask, region_labels_to_join, thickness=5) #thickness is the thickness of the line segment used to join the regions
colored_final = label2rgb(final_mask, bg_label=0)
########################################################################################################
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Visualization:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.imshow(label2rgb(labeled_full_binary_mask, bg_label=0))
plt.title(textwrap.fill("Original Labeled Mask", width=25))
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(colored_final)
plt.title(textwrap.fill("Final Labeled Mask (Regions Joined)", width=25))
plt.axis('off')
plt.show()


EXTRACT CONTOUR AND FIT A SMOOTHING SPLINE TO THE CUSPS CONTOUR

In [21]:
#############################################################################################################
# IDENTIFY AND PLOT ALL CONTOURS IN THE FINAL MASK. NOTE THE CONTOUR ID THAT BEST DEPICTS THE CUSPS PROFILE
############################################################################################################

# 1. Detect All Contours
contours = measure.find_contours(final_mask, level=0.5)
if not contours:
    raise ValueError("No contours found. Check the binary mask.")

# 2. Plot All Contours in Different Colors
plt.figure(figsize=(18, 8))
plt.imshow(final_mask, cmap='gray', origin='upper')
plt.title("All Detected Contours")
plt.axis('off')

colors = cm.rainbow(np.linspace(0, 1, len(contours)))
for i, c in enumerate(contours):
    # c is an Nx2 array [row, col]
    x_c = c[:, 1]  # col -> x
    y_c = c[:, 0]  # row -> y
    
    # If the contour is closed, remove the duplicate end point
    if np.allclose(c[0], c[-1]):
        x_c = x_c[:-1]
        y_c = y_c[:-1]
    
    plt.plot(x_c, y_c, color=colors[i], linewidth=3, label=f"Contour {i+1}")

plt.axis('off')
# Place the legend to the right side of the plot
plt.legend(loc='center left', ncols = 3, bbox_to_anchor=(1, 0.5), borderaxespad=0.4)

plt.tight_layout()
plt.show()

print("Plotted", len(contours), "contours. Inspect visually to decide which one you want and indicate the index in the next code cell.")


Plotted 1 contours. Inspect visually to decide which one you want and indicate the index in the next code cell.


In [23]:
###############################################################################################
# SELECT THE CONTOUR THAT BEST DEFINE THE CUSPS PROFILE AND FIT THE SMOOTHING SPLINE
###############################################################################################

# !!!!!!!!!!!!!!!!!!!!!!!!!!!

# Suppose the user picks an index after inspection
chosen_idx = 0  # for example, user says "I want Contour 1"

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Extract that contour
best_contour = contours[chosen_idx]

# Convert row->y, col->x
x_cont = best_contour[:, 1]
y_cont = best_contour[:, 0]

# If it's closed, remove the last point
if np.allclose(best_contour[0], best_contour[-1]):
    x_cont = x_cont[:-1]
    y_cont = y_cont[:-1]

# Downsample if desired (downsampling reduces the number of points in the contour and enahnces the smoothing spline)
downsample_factor = 10
x_down = x_cont[::downsample_factor]
y_down = y_cont[::downsample_factor]

# Parameterize by arc length
diffs = np.diff(np.column_stack((x_down, y_down)), axis=0)
seg_lengths = np.sqrt((diffs**2).sum(axis=1))
arc_length = np.concatenate([[0], np.cumsum(seg_lengths)])
arc_length /= arc_length[-1]

# Fit the spline
smoothing_factor = 800
tck, u = splprep([x_down, y_down], u=arc_length, s=smoothing_factor)
u_new = np.linspace(0, 1, 10000)
x_smooth, y_smooth = splev(u_new, tck)

# Plot the chosen contour + the final smoothed spline
plt.figure(figsize=(12, 10))
plt.imshow(final_mask, cmap='gray', origin='upper')
plt.title("Fitted spline to cusps contour")

# Show the downsampled contour
plt.plot(x_down, y_down, 'bo', markersize=5, label="Downsampled Points")

# Show the final spline
plt.plot(x_smooth, y_smooth, 'r-', linewidth=5, label='Smoothed Spline')

plt.axis('off')
plt.legend(loc='center left', bbox_to_anchor=(.85, 0.5))
plt.show()


POST-PROCESSING CUSPS PROFILE (REMOVING UNWANTED LINE SEGMENTS)

In [25]:
##############################################
##############################################

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# ONLY USE THIS SECTION IF FURTHER POST-PROCESSING (LIKE HERE BREAKING OFF EXTRA SEGMENTS) ARE REQUIRED

#################################################
#################################################

# -------------------------------
# Part 1: Break the Profile at the First and Last Peak
# -------------------------------

# (A) Detect the peak points.
# For this example, we assume peaks are the local maxima in x_smooth.
# (Adjust the prominence parameter as needed.)
peak_indices, _ = find_peaks(x_smooth)

if len(peak_indices) == 0:
    raise ValueError("No peaks were detected in the profile.")

# Identify the first and last peaks.
first_peak = peak_indices[2]
last_peak = peak_indices[-1]

# Restrict the profile to the segment between the first and last peaks.
x_profile = x_smooth[first_peak:last_peak+1]
y_profile = y_smooth[first_peak:last_peak+1]

# (Optional) Visualize the full profile with peaks and the restricted segment.
plt.figure(figsize=(10,6))
plt.imshow(final_mask, cmap='gray', origin='upper')
plt.plot(x_smooth, y_smooth, 'y-', linewidth=4, label='Full Profile')
plt.plot(x_smooth[peak_indices], y_smooth[peak_indices], 'bo', markersize=10, label='Initial Peaks')
plt.plot(x_profile, y_profile, 'r-', linewidth=4, label='Acutal Profile')
plt.xlabel("Easting (m)")
plt.ylabel("Northing (m)")
plt.title("Modified Profile")
plt.legend(loc='center left', bbox_to_anchor=(.85, 0.5))
plt.show()


DETECT PEAKS AND DETERMINE CUSPS SPACING

In [ ]:
CONVERT PIXEL COORDINATES OF CUSPS PROFILE TO SPATIAL COORDINATES AND CONVERT PROFILE TO SHAEFILE

In [27]:
###############################################################################################
# CONVERT PIXEL COORDINATES OF SMOOTH CURVE TO REAL WORL COORDINATES OF IMAGE
################################################################################################

transform = src.transform
crs = src.crs
width, height = src.shape

# compute the x/y extent in case we want to display the mask in proj coords
left, top = transform * (0, 0)
right, bottom = transform * (width, height)
extent = (left, right, bottom, top)

def pixel_to_real_world_coords(col_array, row_array, aff_transform):
    """
    Convert arrays of pixel coordinates (col, row) to real-world coords (x, y)
    using the given affine transform from rasterio.
    """
    x_coord = []
    y_coord = []
    for c, r in zip(col_array, row_array):
        x, y = aff_transform * (c, r)
        x_coord.append(x)
        y_coord.append(y)
    return np.array(x_coord), np.array(y_coord)

############################################
# 3) Convert Your Smoothed Curve to Real-World
############################################
# x_smooth, y_smooth = ...  # from your final wavy curve in pixel coords
# binary_mask = ...         # 2D array in pixel coords

# Convert the smoothed curve to real-world coords
x_curve, y_curve = pixel_to_real_world_coords(x_profile, y_profile, transform)


#######################################################
# CONVERT PROFILE TO SHAPEFILE
####################################################

# Create a LineString geometry from the spatial coordinates.
cusps_line = LineString(zip(x_curve, y_curve))

# Define a simple schema for a LineString feature.
schema = {
    'geometry': 'LineString',
    'properties': {}
}

# Write the geometry to a shapefile.
with fiona.open('cusps_profile.shp', 'w', driver='ESRI Shapefile',
                crs=src.crs, schema=schema) as shp:
    shp.write({
        'geometry': mapping(cusps_line),
        'properties': {}
    })

print("Shapefile 'cusps_profile.shp' created with the same spatial information as the source image.")

Shapefile 'cusps_profile.shp' created with the same spatial information as the source image.


PEAKS DETECTION

In [29]:
################################################################################################
# DETECT HIGH PEAKS IN THE SMOOTH CURVE
###############################################################################################

# Adjust based on how many peaks you want to detect
peaks, _ = find_peaks(x_curve, prominence = .1)

# Compute the real-world extent of the mask
height, width = final_mask.shape
# Top-left corner in UTM
x_min, y_max = transform * (0, 0)
# Bottom-right corner in UTM
x_max, y_min = transform * (width, height)

# extent = (left, right, bottom, top)
mask_extent = (x_min, x_max, y_min, y_max)

# -------------------------
# Step 2: Visualization in the Image's Coordinate System
# -------------------------
fig, ax = plt.subplots(figsize=(12,10))

# If you have a binary mask or original image to display in pixel coords:
ax.imshow(final_mask, cmap='gray', extent = mask_extent)
ax.set_title(textwrap.fill("Smoothed Curve and Detected Peaks", width=25))
# Plot the smooth curve
ax.plot(x_curve, y_curve, 'r-', linewidth=5, label='Smoothed Curve')

# Plot the detected peaks
ax.plot(x_curve[peaks], y_curve[peaks], 'bo', markersize=10, label='Detected Peaks')

ax.set_xlabel("Eastings (m)", fontsize=25)
ax.set_ylabel("Northings (m)", fontsize=25)
plt.legend(loc='center left', bbox_to_anchor = (0.82,0.5), fontsize=25)

# Format axes to display normal values without scientific notation or decimals
ax.xaxis.set_major_locator(MaxNLocator(2, integer=True))
ax.yaxis.set_major_locator(MaxNLocator(4, integer=True))

# Format axes ticks to plain style, zero decimals
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

plt.show()

print(f"Detected {len(peaks)} peaks at indices: {peaks}")


Detected 6 peaks at indices: [ 922 1628 2515 3734 4374 4802]


ESTIMATE CUSPS SPACING

In [33]:
###############################################################################################
# CALCULATE CUSPS SPACING FROM DETECTED PEAKS
###############################################################################################

# -------------------------
# Step 1: Extract Peak Coordinates
# -------------------------
x_peaks = x_curve[peaks]
y_peaks = y_curve[peaks]

# -------------------------
# Step 2: Compute Euclidean Distances Between Consecutive Peaks
# -------------------------
distances = []
for i in range(len(x_peaks) - 1):
    dx = x_peaks[i+1] - x_peaks[i]
    dy = y_peaks[i+1] - y_peaks[i]
    dist = math.sqrt(dx**2 + dy**2)
    distances.append(dist)

# Create a DataFrame for cusp spacing
cusp_ids = [f"Cusp {i+1}" for i in range(len(distances))]
SAR_cusps_spacing = pd.DataFrame({
    "Cusp ID": cusp_ids,
    "Spacing": distances
})

print("Cusp Spacing DataFrame:")
display(SAR_cusps_spacing)

# -------------------------
# Step 3: Final Visualization (Marking the First Cusp with X)
# -------------------------

fig, ax = plt.subplots(figsize=(12,10))

ax.imshow(final_mask, cmap='gray', extent = mask_extent)
# plt.figure(figsize=(12, 10))
# plt.imshow(final_mask, cmap='gray', extent = mask_extent)
# ax.set_title(textwrap.fill("Final Plot with First Horn Marked", width=25))

# Plot the smooth curve
ax.plot(x_curve, y_curve, 'r-', linewidth=4, alpha=1, label='Cusps Profile')

# Plot all peaks in blue
ax.plot(x_peaks, y_peaks, 'bo', markersize=15, alpha=1, label='All Peaks')

# Mark the first cusp with an X (this is x_peaks[0], y_peaks[0])
ax.plot(x_peaks[0], y_peaks[0], 'X', color='yellow', markersize=20, alpha=1, label='First Horn')


# 1) Set your locators
ax.xaxis.set_major_locator(MultipleLocator(35))
ax.yaxis.set_major_locator(MultipleLocator(60))

# 2) Install your zero‐decimal formatter
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

# 3) Hide any leftover “offset” text
ax.xaxis.get_offset_text().set_visible(False)
ax.yaxis.get_offset_text().set_visible(False)

# 4) (Optional) Rotate y‐ticks if you like, but they’ll stay perfectly centered
for lbl in ax.yaxis.get_ticklabels():
    lbl.set_rotation(90)
    lbl.set_verticalalignment('center')

ax.set_xlabel("Eastings (m)", fontsize=30)
ax.set_ylabel("Northings (m)", fontsize=30)
ax.tick_params(axis='both', which='major', length=7, labelsize=25)
plt.axis('on')
# plt.legend(loc='center left', bbox_to_anchor = (0.82,0.5), fontsize=25)


plt.tight_layout()
plt.savefig('SAR_Cusp_Profile.jpeg', bbox_inches='tight')
plt.show()

Cusp Spacing DataFrame:


,Cusp ID,Spacing
0,Cusp 1,25.841682
1,Cusp 2,27.265248
2,Cusp 3,25.188091
3,Cusp 4,25.474018
4,Cusp 5,19.600885
